# 1. Setup and Installation

In [1]:
# ! pip install numpy pandas scikit-learn matplotlib seaborn

# 2. Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.datasets import load_iris

# 3. Loading the Data

In [96]:
# Load dataset
iris = load_iris()

X = iris.data
y = iris.target

# since we are doing binary classifcation , we'll only use two classes (0 and 1)
X = X[y != 2]#[:,2]
y = y[y != 2]

In [97]:
# Check the shape of the data
print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

Feature matrix shape: (100, 4)
Target vector shape: (100,)


In [98]:
np.unique(y, return_counts=True)

(array([0, 1]), array([50, 50]))

# 4. Splitting the Data

In [99]:
# Split the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Reshape if the data is 1D (single feature per sample)
# X_train = X_train.reshape(-1, 1)
# X_test = X_test.reshape(-1, 1)


# Check the shape of the train/test split
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (70, 4)
Testing set shape: (30, 4)


# 5. Feature Scaling

In [102]:
# Standardizing the features
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 6. Train the Logistic Regression Model

In [103]:
# Creating a logistic regression classifier
log_reg = LogisticRegression()

In [104]:
# Fit the model on the training data
log_reg.fit(X_train, y_train)

LogisticRegression()

In [105]:
# Predict on test set
y_pred = log_reg.predict(X_test)

In [106]:
# check accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")

Accuracy: 1.0000


# 7. Evaluation of the Model

In [107]:
# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("COnfusion Matrix:")
print(conf_matrix)

# Classification report (precision, recall, f1-score)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Accuracy
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

COnfusion Matrix:
[[17  0]
 [ 0 13]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       1.00      1.00      1.00        13

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

Accuracy: 1.0000
